# Udacity Data Engineering Nanodegree - Project 2: Data Modelling in Apache Cassandra

This is my solution for the second proect of the [Udacity Data Engineering Nanodegree](https://eu.udacity.com/course/data-engineer-nanodegree--nd027). It differs slightly from the suggested notebook structure, still I believe all relevant specifications are met.

## Setup

First, we set the working directory and load all necessary modules.

In [1]:
import sys
sys.path.append("../data_modeling/")

In [2]:
import answer
import create_event_data_new
import create_tables
import cql_queries
import etl

## Part 1: ETL Pipeline for Preprocessing the Files

In part 1, we merge all provided files in `data/event_data` to one file `event_data_new.csv`.

In [3]:
create_event_data_new.main()

Reading event_data files...
Writing event_data_new file...
Done.


## Part 2: Apache Cassandra

In part 2, we create the database and tables, run the ETL to populate the tables and query the tables to answer our business questions. The questions we are trying to answer are:

1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Creating Database and Tables

The code to create database and tables lies in `data_modelling/create_tables.py`. In order to answer the three questions above, the module creates three tables for us: `song_in_session`, `song_in_user` and `user_in_song`.

In [17]:
for query in cql_queries.CREATE_TABLE_QUERIES:
    print(query)


CREATE TABLE IF NOT EXISTS song_in_session (
                                    session_id INT,
                                    item_in_session INT,
                                    artist_name VARCHAR,
                                    song_title VARCHAR,
                                    song_duration DECIMAL,
                                    PRIMARY KEY(session_id,
                                                item_in_session)
                                    );


CREATE TABLE IF NOT EXISTS song_in_user (
                                    user_id INT,
                                    session_id INT,
                                    item_in_session INT,
                                    artist_name VARCHAR,
                                    song_title VARCHAR,
                                    user_first_name VARCHAR,
                                    user_last_name VARCHAR,
                                    PRIMARY KEY(user_id,
                

In [5]:
create_tables.main()

Creating connection...
Dropping old tables...
Creating new tables...
Closing connection...
Done.


### Inserting the Data into Cassandra

The code to run the ETL and insert data into our tables lies in `data_modelling/etl.py`. For every table, raw data is preprocessed as described in `data_modelling/etl_steps/prepare.py`. Afterwards, the script inserts the data as follows:

In [6]:
for query in cql_queries.INSERT_TABLE_QUERIES:
    print(query)


INSERT INTO song_in_session (
                     session_id,
                     item_in_session,
                     artist_name,
                     song_title,
                     song_duration)
VALUES (
        %(session_id)s,
        %(item_in_session)s,
        %(artist_name)s,
        %(song_title)s,
        %(song_duration)s
        );


INSERT INTO song_in_user (
                     user_id,
                     session_id,
                     item_in_session,
                     artist_name,
                     song_title,
                     user_first_name,
                     user_last_name)
VALUES (
        %(user_id)s,
        %(session_id)s,
        %(item_in_session)s,
        %(artist_name)s,
        %(song_title)s,
        %(user_first_name)s,
        %(user_last_name)s
        );


INSERT INTO user_in_song (
                     song_title,
                     user_id,
                     user_first_name,
                     user_last_name)
VALUES (


In [7]:
etl.main()

Creating connection...
Inserting data...
Closing connection...
Done.


### Answering the Business Questions

Finally we can answer our business questions! In order to do so, we execute the following queries and store the results in `answers/` in markdown format.

In [8]:
for query in cql_queries.SELECT_TABLE_QUERIES:
    print(query)


SELECT artist_name,
       song_title,
       song_duration
  FROM song_in_session
 WHERE session_id=338
   AND item_in_session=4;


SELECT artist_name,
       song_title,
       user_first_name,
       user_last_name
  FROM song_in_user
 WHERE user_id=10
   AND session_id=182;


SELECT user_first_name,
       user_last_name
  FROM user_in_song
 WHERE song_title = 'All Hands Against His Own';



In [9]:
answers = answer.main()

Creating connection...
Executing queries...
Closing connection...
Done.


## Final Answers

Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [12]:
answers[0]

,artist_name,song_title,song_duration
0,Faithless,Music Matters (Mark Knight Dub),495.3073


Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [14]:
answers[1]

,artist_name,song_title,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
answers[2]

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson
